In [11]:

import os as os
import csv as csv
import re as re
import gzip as gz
import operator as op

import pandas as pd

folder = '/TL/deep/fhgfs/projects/pebert/thesis/projects/statediff/references'

inputfile = os.path.join(folder, 'raw', 'ucsc_refseq_func-elem.bed.gz')
outputfile = os.path.join(folder, 'UCSC_hg38_RefSeqFuncElem_1-22X.bed')

chrom_re = re.compile('chr[0-9X]+(\s|$)')
buffer = []
counter = col.Counter()
get_record = op.itemgetter(*('chrom', 'chromStart', 'chromEnd', 'soTerm', 'score', 'strand'))
with gz.open(inputfile, 'rt') as tsv:
    header = tsv.readline().strip().split('\t')
    header[0] = 'chrom'
    rows = csv.DictReader(tsv, delimiter='\t', fieldnames=header,
                          dialect=csv.unix_dialect)
    for row in rows:
        if chrom_re.match(row['chrom']) is not None:
            buffer.append(get_record(row))
        
df = pd.DataFrame.from_records(buffer, columns=['chrom', 'start', 'end', 'name', 'score', 'strand'])
df.sort_values(['chrom', 'start'], inplace=True)
df['score'] = df['end'].astype('int32') - df['start'].astype('int32')
df['score'].clip(0, 1000, inplace=True)

with open(outputfile, 'w') as dump:
    _ = dump.write('#')
    df.to_csv(dump, sep='\t', index=False, header=True)
